In [93]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.axes3d import Axes3D
import time

In [94]:
X = np.array([[ 1,  1],
              [ 1,  1],
              [ 1,  2],
              [ 1,  5],
              [ 1,  3],
              [ 1,  0],
              [ 1,  5],
              [ 1, 10],
              [ 1,  1],
              [ 1,  2]])

In [13]:
y = [45, 55, 50, 55, 60, 35, 75, 80, 50, 60]

**1. Подберите скорость обучения (eta) и количество итераций**

In [112]:
def calc_mae(y, y_pred):
    err = np.mean(np.abs(y - y_pred))
    return err

def calc_mse(y, y_pred):
    err = np.mean((y - y_pred)**2)
    return err

def mserror(X, w, y):
    y_pred = X.dot(w)
    return (np.sum((y_pred - y)**2)) / len(y)

In [96]:
n = X.shape[0]

eta = 0.1 #меняю коэффициент
n_iter = 150 #меняю коэффициент

W = np.array([1, 0.5])
print(f'Number of objects = {n} \
       \nLearning rate = {eta} \
       \nInitial weights = {W} \n')

#Добавил отражение времени выполнение скрипта

Number of objects = 10        
Learning rate = 0.1        
Initial weights = [1.  0.5] 



In [97]:
%%time 

for i in range(n_iter):
    y_pred = np.dot(X, W)
    err = calc_mse(y, y_pred)
    for k in range(W.shape[0]):
        W[k] -= eta * (1/n * 2 * X[:, k] @ (y_pred - y))
    if i % 10 == 0:
        eta /= 1.1
        print(f'Iteration #{i}: W_new = {W}, MSE = {round(err, 2)}')

Iteration #0: W_new = [11.8 38.2], MSE = 3047.75
Iteration #10: W_new = [12651.73553914 69617.0969639 ], MSE = 18310954068.05
Iteration #20: W_new = [ 7732434.81888021 42641607.37852186], MSE = 9128819654907568.0
Iteration #30: W_new = [1.06344502e+09 5.86454589e+09], MSE = 2.327920364266843e+20
Iteration #40: W_new = [3.00127077e+10 1.65510116e+11], MSE = 2.545133529815933e+23
Iteration #50: W_new = [1.55345341e+11 8.56677968e+11], MSE = 9.572295620500074e+24
Iteration #60: W_new = [1.27742291e+11 7.04456313e+11], MSE = 9.351480126475957e+24
Iteration #70: W_new = [1.38141953e+10 7.61806995e+10], MSE = 1.640858952828383e+23
Iteration #80: W_new = [1.51674189e+08 8.36432543e+08], MSE = 3.125533537874439e+19
Iteration #90: W_new = [116395.49988139 641638.79864291], MSE = 31317286806394.04
Iteration #100: W_new = [48.41966454 22.99883908], MSE = 53518.86
Iteration #110: W_new = [44.9771605   3.82798314], MSE = 43.97
Iteration #120: W_new = [44.99895429  3.82402303], MSE = 43.97
Iteration

**Вывод**

Достаточная точность достигается при скорости 0.1 и количестве итераций 150

Увеличение кол-ва итераций приводит к пропорциональному увеличению времени работы скрипта (16ms при 150 и 34.5 ms при 300).

**2*. В этом коде мы избавляемся от итераций по весам, но тут есть ошибка, исправьте ее**

In [99]:
for i in range(n_iter):
    y_pred = np.dot(X, W)
    err = calc_mse(y, y_pred)
#     for k in range(W.shape[0]):
#         W[k] -= eta * (1/n * 2 * X[:, k] @ (y_pred - y))
    # ИЗМЕНЕНИЯ
    
    W -= eta * 2/n * np.dot(X.T, y_pred - y) # ИЗМЕНЕНИЯ: транспонируем
    
    # ИЗМЕНЕНИЯ
    #
    if i % 10 == 0:
        print(f'Iteration #{i}: W_new = {W}, MSE = {round(err,2)}')

Iteration #0: W_new = [45.06138476  3.81270223], MSE = 43.97
Iteration #10: W_new = [45.06160566  3.81266217], MSE = 43.97
Iteration #20: W_new = [45.06178281  3.81263005], MSE = 43.97
Iteration #30: W_new = [45.06192487  3.81260429], MSE = 43.97
Iteration #40: W_new = [45.06203879  3.81258363], MSE = 43.97
Iteration #50: W_new = [45.06213014  3.81256707], MSE = 43.97
Iteration #60: W_new = [45.0622034   3.81255378], MSE = 43.97
Iteration #70: W_new = [45.06226215  3.81254313], MSE = 43.97
Iteration #80: W_new = [45.06230926  3.81253459], MSE = 43.97
Iteration #90: W_new = [45.06234704  3.81252774], MSE = 43.97
Iteration #100: W_new = [45.06237734  3.81252224], MSE = 43.97
Iteration #110: W_new = [45.06240164  3.81251784], MSE = 43.97
Iteration #120: W_new = [45.06242112  3.8125143 ], MSE = 43.97
Iteration #130: W_new = [45.06243674  3.81251147], MSE = 43.97
Iteration #140: W_new = [45.06244927  3.8125092 ], MSE = 43.97


**3*. Вместо того, чтобы задавать количество итераций, задайте другое условие останова алгоритма - когда веса перестают изменяться меньше определенного порога $\epsilon$.**

In [118]:
w_list = [W.copy()]

min_weight_dist = 1e-8

weight_dist = np.inf

errors = []

iter_num = 0

In [119]:
while weight_dist > min_weight_dist:
    y_pred = np.dot(X, W)
    dQ = 2/n * X.T @ (X @ W - y)
    new_w = W - eta * dQ
    weight_dist = np.linalg.norm(new_w - W, ord=2)
    error = mserror(X, new_w, y)
    
    w_list.append(new_w.copy())
    errors.append(error)
    
    print(f'Iter {iter_num}: error - {error}, weights: {new_w}')
    
    iter_num += 1
    W = new_w
    
w_list = np.array(w_list)
w_pred = w_list[-1]

print(f'В случае использования градиентного спуска функционал ошибки составляет {round(errors[-1], 4)}')
    


Iter 0: error - 43.96875000077936, weights: [45.06245932  3.81250738]
Iter 1: error - 43.968750000745686, weights: [45.06246021  3.81250722]
Iter 2: error - 43.968750000713484, weights: [45.06246108  3.81250706]
Iter 3: error - 43.968750000682675, weights: [45.06246193  3.8125069 ]
Iter 4: error - 43.96875000065319, weights: [45.06246276  3.81250675]
Iter 5: error - 43.968750000624986, weights: [45.06246357  3.81250661]
Iter 6: error - 43.96875000059799, weights: [45.06246437  3.81250646]
Iter 7: error - 43.968750000572165, weights: [45.06246515  3.81250632]
Iter 8: error - 43.96875000054745, weights: [45.06246591  3.81250618]
Iter 9: error - 43.96875000052382, weights: [45.06246665  3.81250605]
Iter 10: error - 43.96875000050119, weights: [45.06246738  3.81250592]
Iter 11: error - 43.968750000479545, weights: [45.06246809  3.81250579]
Iter 12: error - 43.968750000458826, weights: [45.06246879  3.81250566]
Iter 13: error - 43.968750000439016, weights: [45.06246947  3.81250554]
Iter 14: